# Obtener funciones

In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:

import Integral as In
import numpy as np
from scipy.integrate import dblquad

In [22]:
h = 0.001
f = lambda x: ((x+2)*(x-2))/((x-3)*(x-4))

In [23]:
In.D_central(6,h,f)

-2.4444446165112765

In [37]:
f(6)

5.333333333333333

In [41]:
l=In.I_trapecio([0,1],lambda x: np.exp(-x**2),8)
print(l)

0.7455719918300938


In [55]:
def f(x,y):
    return x*y

f(3,4)


12

In [102]:
def RecBonnet(n,x):
    p0, pn = 1,x
    for j in range(1,n):
        pn1 = ((2*j+1)*x*pn - j*p0)/(j+1.0)
        p0,pn = pn,pn1
    return p0,pn

def legendre(n, x):
    if n==0:
        valn = 1
        dvaln = 0
    elif n==1:
        valn = x
        dvaln = 1
    else:
        valn_m1, valn = RecBonnet(n, x)
        dvaln = n*(valn_m1-x*valn)/(1.-x**2)
    return valn, dvaln


def legraiz(n, delt=.2, error='E_rel2', tol=1e-5):
    raices = np.zeros(n)
    npos = n//2 
    
    f = lambda x: legendre(n, x)[0]  
    for i in range(npos):
        p0 = np.cos(np.pi*(4*i+3)/(4*n+2)) - delt 
        p1 = p0 + 2*delt
        print('p0 -->', p0,'p1-->', p1)
        raiz = rr.secante2([p0, p1], f, error, tol)
        raices[i] = -raiz
        raices[-1-i] = raiz
    return raices

def Gus_param(n, delt=0.2, error='E_rel2', tol=1e-05): 
    xraiz = legraiz(n, delt, error, tol)
    Pn = legendre(n, xraiz)[1]
    W = 2/(1-xraiz**2)/Pn**2
    return xraiz, W
Gus_param(4)

p0 --> 0.6660254037844386 p1--> 1.0660254037844386
p0 --> 0.1420201433256688 p1--> 0.5420201433256688


(array([-0.86113631, -0.33998104,  0.33998104,  0.86113631]),
 array([0.34785485, 0.65214515, 0.65214515, 0.34785485]))

In [4]:
def I_Gauss2D(I2,I,f,n = 2):
    """
    Integral doble Gauss
    I: Intervalo de y, con funciones.
    I2: Intervalo de x
    f: la funcion f(x,y)
    n: El gadro del polinomio de Legender
    """
    D2 = 0
    xr,W = Gus_param(n, delt=.2, error='E_rel2', tol=1e-05)
    for i in range(1,n):
        for j in range(1,n):
            u2 = (I2[1]+I2[0])/2 + ((I2[1]-I2[0])*xr[j])/2
            u = (I[1]+I[0])/2 + ((I[1]-I[0])*xr[i])/2
            D2 = D2 + (W[j]**2)*f(u2,u)
    return D2*(I2[1]-I2[0])*(I[1]-I[0])/2
print(I_Gauss2D([0,1],[1,3], lambda x,y : 2*x*y, n=2))
#print(I_Gauss2D([0,1.21341166],[lambda x: 3*x - x**2, lambda x: x**4], lambda x,y : x, n=2))



inf


/tmp/ipykernel_2514/1122634417.py:37: RuntimeWarning: divide by zero encountered in divide
  W = 2/(1-xraiz**2)/Pn**2


In [6]:
print(In.I_Gauss([1.,1.],[1,3],[-0.57735027 ,0.57735027], lambda x: x,2))
def I_Gauss(c,a,xr, f,n =2):
    I_G = 0
    for i in range(0,n):
        x = (a[1]+a[0])/2 + (a[1]-a[0])*xr[i]/2
        I_G = I_G + c[i]*f(x)
        
    return ((a[1]-a[0])*I_G)/2
I_Gauss([1.,1.],[1,3],[-0.57735027 ,0.57735027], lambda x: x,2)

2.57735027


4.0

# Dividir intervalos

In [4]:
def es_flotante(variable):
    """
    -----------------------
    Verificador de numeros
    ------------------------
    
    Parametros:
    float: una valor de tipo int o flotante
    
    -----------------------------------
    Salida: Regresa un verdadero si cumple con un numero y un falso si es un caracter
    ----------------------------------
    """
    try:
        float(variable)
        return True

    except:
        return False

#Meter intervalo

def intr_int():
    """
    --------------------
    Ingresar Parametros
    --------------------

    Parametros:
    a: El intervalo menor
    b: El intervalos mayor
    
    -----------------------
    Salida: Regresa una lista con los intervalos
    -----------------------
    """
    
    a = input("Ingrese el intervalo izquierdo: ")
    b = input("Ingrese el intervalo derecho: ")
    return [a,b]


#Esta funcion se encarga de ver que el producto de las dos funciones se menor que cero

def sigma(funcion, x, tol = 1e-4):
    """
    --------------------------------------
    Retrorno de sigo evaluado en funcion
    --------------------------------------
    
    Parametros:
    f: Una funcion lambda o def, la cual sera nuestra función
    x: Nuestras variables que se encargaran de evaluar nuestra función

    -------------------------------------------------------------
    Salida: Regresa -1 si f(a)*f(b) es menor y 1 si es mayor
    -------------------------------------------------------------
    """

    a, b = x[0], x[1]
    sigm = 1 if funcion(a)*funcion(b) > 0 else -1
    if funcion(a)*funcion(b) > 0:
        sigm = 1
    elif (funcion(a)*funcion(b) == 0 ) or funcion(a)*funcion(b) <= tol:
        sigm = 0
    else: 
        sigm = -1
    return sigm

#Esta se encarga de guardar los intervalos donde exista una posible raiz


def localizador(f,x0,tol = 1e-4, Ni = 100):
    """
    ----------------------------------------------------
    Busca las posibles raices en una funcion
    ----------------------------------------------------
    Parametros:
    f: Funcion lambda o def, f(x).
    intervalo: los extremos del intervalo a analizar
    Nint: La cantidad de puntos a partir

    ----------------------------------------------------------------------------------------------
    Salida: Una lista que aguarda varias posiciones para nuestras posibles raices, la cual alberga otra lista con dos posiciones donde se encuentra una raiz
    ----------------------------------------------------------------------------------------------------
    
    """
    x1 = (x0[1]-x0[0])/Ni
    x2 = []
    x3 = x0[0] + x1
    x4 = x3
    x2.append(x3)
    for i in range(Ni):
        x4 = x4 + x1
        x2.append(x4)
    localiz = []
    for i in range(1, Ni):
        comparar = [x2[i-1],x2[i]]
        if sigma(f, comparar, tol) < 0 or sigma(f, comparar, tol) == 0:
            localiz.append(comparar)
    return localiz


In [80]:
rr.raiz_N([-7,7], lambda x: (1-np.cos(x))/x**2)
rr.raiz_secante1([0,4], lambda x: x**2-7*x+12)
rr.raiz_secante2([-2,2], lambda x: x**2-3)

[-1.732050807559437, 1.7320508075905212]

In [26]:
def secante2(x,funcion, errores = 'E_ab',tol = 1e-4):
    """
    El metodo de la secante para una sola raiz
    
    Los parametros que cuenta:
    x: Recibe dos parametros, usualmete los extremos del intervalo [a,b] a analizar.
    f: La funcion lambda o el objeto def , estos refresentan la funcion que se analizara.

    errores: Tipos de errores, E_ab (error absoluto), E_rel (error relativo), 
    E_RPD (error diferencia de procentaje relativo), E_rel3
    (error de diferencia y cambio relativo), E_dis (error de sitancia), E_rel2 (error relativo para convergencias cercanas a cero).
    
    tol: la presicion a buscar, por defecto es en 1e-4

    ------------------------
    Salida: raiz
    ------------------------

    """
    if sigma(funcion, x) == -1 or sigma(funcion, x) == 0:
        operador = True
        raiz = 0
        if es_flotante(x[0]) == True and es_flotante(x[1]) == True:    
            w = [float(x[i]) for i in range(2)]
            while operador:
                p = w[1]-(funcion(w[1]) * (w[1] - w[0]))/(funcion(w[1])-funcion(w[0]))
                if error(errores)(w[0],w[1]) < tol:
                    #print("La raiz de su función es: ", p)
                    raiz = p
                    operador = False
                w[0] = w[1]
                w[1] = p
            return raiz
        else:
            print("Valores erroneos")
    else:
        raiz = None
        return raiz
def raiz_secante2(x1, f, errores = 'E_ab', tol = 1e-4):
    """
    El metodo de la secante 2 para una o más raices

    Los parametros que cuenta:
    x1: Recibe dos parametros, usualmete los extremos del intervalo [a,b] a analizar.
    f: La funcion lambda o el objeto def , estos refresentan la funcion que se analizara.

    errores: Tipos de errores, E_ab (error absoluto), E_rel (error relativo), 
    E_RPD (error diferencia de procentaje relativo), E_rel3
    (error de diferencia y cambio relativo), E_dis (error de sitancia), E_rel2 (error relativo para convergencias cercanas a cero).
    
    tol: la presicion a buscar, por defecto es en 1e-4

    ------------------------
    Salida: raices
    ------------------------
    """
    if es_flotante(x1[0]) == True and es_flotante(x1[1]) == True:
        w = [float(x1[i]) for i in range(2)]
        raiz = []
        if sigma(f,w) == -1 or sigma(f,w) == 0: 
            raiz_loop = localizador(f, w)
            for i in raiz_loop:
                p = secante_2(i, f, errores, tol)
                raiz.append(p)
            return raiz
        else:
            raiz.append(None)
            return raiz
    else:
        print("Valores erroneos")
        

In [110]:
import Raiz as rr
#np.poly(2,-6,10,-6)
l = rr.raiz_secante2([0,4], lambda x: x**2-1)
print(l)

[1.0]
